# Tools

Tool calling is the ability of a model to determine when to call an external function and structure data for this call.
Essentially, it creates a bridge between the text capabilities of LLMs and external tools or APIs.

**How it works**:
1. The model identifies when a user request requires using a tool
2. The LLM transforms the necessary parameters into JSON or another structured format
3. The external system executes the tool with the received parameters
4. The tool execution results are returned to the model, which incorporates them into its response

This approach expands LLM capabilities both in obtaining data and performing actions in specific systems.
For example,
you can write a function that retrieves current weather information,
allowing the LLM to provide clothing recommendations.
Or you could provide tools for interacting with SQL databases,
enabling natural language queries that execute the necessary SQL commands behind the scenes.

Let's add our dependencies,
API key,
and define a `ChatClient` as we did in previous notebooks.

In [1]:
%useLatestDescriptors
%use spring-ai-openai

In [2]:
val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

val openAiApi = OpenAiApi.builder().apiKey(apiKey).build()
val openAiOptions = OpenAiChatOptions.builder()
    .model(OpenAiApi.ChatModel.GPT_4_O_MINI)
    .temperature(0.7)
    .build()

val chatClient = ChatClient.create(
    OpenAiChatModel.builder()
        .openAiApi(openAiApi)
        .defaultOptions(openAiOptions)
        .build()
)

Spring-AI supports tool calling by providing various abstractions for this purpose.
We'll look at two ways to write tools and pass them to an LLM.

The principle of tool calling is well demonstrated by this diagram from the
[Spring-AI documentation](https://docs.spring.io/spring-ai/reference/1.0/api/tools.html#_overview)

<img src="images/framework-manager.jpg" title="Spring AI Tool Calling" height="1008" width="1762"/>

Let's create a GardenTools class with three functions:
- `identifyPlant` — identifies plants by color and key characteristics
- `getPlantingDates` — provides optimal planting dates for various plants
- `analyzeSoil` — provides soil data analysis

In [3]:
class GardenTools {

    @Tool(description = "Identifies plants based on visual characteristics like color, shape, and texture")
    fun identifyPlant(
        @ToolParam(description = "Main color of flowers or leaves") color: String,
        @ToolParam(description = "Distinctive features like fuzzy leaves, thorns, etc") features: String
    ): String = when {
        color == "purple" && features.contains("fuzzy") ->
            """{"name": "Lamb's Ear", "scientificName": "Stachys byzantina", "isInvasive": false}"""

        color == "yellow" && features.contains("tall") ->
            """{"name": "Sunflower", "scientificName": "Helianthus annuus", "isInvasive": false}"""

        else ->
            """{"name": "Unknown plant", "recommendation": "Describe the color and features of the plant in more detail."}"""
    }

    @Tool(description = "Provides optimal planting dates based on plant type and growing zone")
    fun getPlantingDates(
        @ToolParam(description = "Type of plant (e.g., tomatoes, peppers)") plant: String,
        @ToolParam(description = "USDA hardiness zone (e.g., 7b, 8a)") zone: String
    ): String = when {
        plant.contains("tomato") && zone == "7b" ->
            """{"startDate": "April 15", "endDate": "May 30", "notes": "Wait until soil temperature reaches 60°F"}"""

        plant.contains("pepper") && zone == "7b" ->
            """{"startDate": "May 1", "endDate": "June 15", "notes": "These plants prefer warm soil"}"""

        else ->
            """{"error": "Planting information not available for this combination"}"""
    }

    @Tool(description = "Analyzes soil characteristics and provides recommendations")
    fun analyzeSoil(
        @ToolParam(description = "Type of soil (clay, sandy, loam)") soilType: String,
        @ToolParam(description = "Type of plant being grown") plantType: String
    ): String = when (soilType) {
        "clay" -> """{"pH": 7.2, "drainage": "poor", "amendments": ["compost", "perlite"]}"""
        "sandy" -> """{"pH": 6.5, "drainage": "excessive", "amendments": ["compost", "coconut coir"]}"""
        "loam" -> """{"pH": 6.8, "drainage": "good", "amendments": ["compost"]}"""
        else -> """{"error": "Unknown soil type"}"""
    }
}

Now that our gardening tools are ready,
let's pass them to the LLM and ask a relevant question:

In [4]:
chatClient
    .prompt("I found this plant with purple flowers and fuzzy leaves in my garden. Also, when's the best time to plant tomatoes in zone 7b?")
    .tools(GardenTools())
    .call()
    .content()

The plant you found with purple flowers and fuzzy leaves is likely **Lamb's Ear** (scientific name: *Stachys byzantina*). It's known for its soft, fuzzy leaves and can add beauty to your garden.

As for planting tomatoes in USDA hardiness zone 7b, the best time to plant is between **April 15** and **May 30**. Make sure to wait until the soil temperature reaches **60°F** before planting for optimal growth.

Now let's look at working with function-based tools.

We'll define functions manually.
In a Spring Application, you could do this dynamically through `@Bean`.

Let's create a Recipe Service:

In [5]:
import kotlin.time.Duration
import kotlin.time.DurationUnit

enum class MealType { BREAKFAST, LUNCH, DINNER }
enum class Diet { VEGAN, VEGETARIAN, OMNIVORE, NONE }

data class RecipeRequest(
    val ingredients: List<String>,
    val mealType: MealType = MealType.DINNER,
    val timeAvailable: Int = 50.minutes.toInt(DurationUnit.MINUTES)
)

data class RecipeResponse(
    val name: String,
    val ingredients: List<String>,
    val instructions: List<String>,
    val cookTime: Duration
)

class RecipeService : (RecipeRequest) -> RecipeResponse {
    override fun invoke(request: RecipeRequest): RecipeResponse {
        return when {
            request.ingredients.containsAll(listOf("pasta", "tomato")) -> {
                RecipeResponse(
                    name = "Quick Pasta Pomodoro",
                    ingredients = listOf("pasta", "tomatoes", "garlic", "olive oil", "basil"),
                    instructions = listOf(
                        "Boil pasta according to package instructions.",
                        "Sauté garlic in olive oil, add diced tomatoes and cook 5 minutes.",
                        "Mix with drained pasta and garnish with basil."
                    ),
                    cookTime = 20.minutes
                )
            }

            request.ingredients.containsAll(listOf("rice", "egg")) -> {
                RecipeResponse(
                    name = "Simple Egg Fried Rice",
                    ingredients = listOf("rice", "eggs", "soy sauce", "green onions"),
                    instructions = listOf(
                        "Scramble eggs in a pan.",
                        "Add cooked rice and stir-fry for 3 minutes.",
                        "Season with soy sauce and top with chopped green onions."
                    ),
                    cookTime = 15.minutes
                )
            }

            else -> {
                RecipeResponse(
                    name = "Pantry Surprise Salad",
                    ingredients = request.ingredients + listOf("olive oil", "salt", "pepper"),
                    instructions = listOf(
                        "Combine all available ingredients in a bowl.",
                        "Dress with olive oil, salt, and pepper."
                    ),
                    cookTime = 10.minutes
                )
            }
        }
    }
}

To let the LLM know about our service, we'll create a `FunctionToolCallback`:

In [6]:
val recipeToolCallback = FunctionToolCallback
    .builder("recipeRecommender", RecipeService())
    .description("Get recipe ideas based on ingredients and time available")
    .inputType(RecipeRequest::class.java)
    .build()

Now we just need to pass this `ToolCallback` to the LLM and ask what we can make for dinner:

In [7]:
chatClient.prompt("I have pasta and tomatoes. What can I make for dinner?")
    .tools(recipeToolCallback)
    .call()
    .content()

With pasta and tomatoes, you have the base for a delicious meal! Here are a few ideas:

1. **Spaghetti with Tomato Sauce**: 
   - Cook the pasta according to package instructions. 
   - In a pan, sauté garlic and onions in olive oil, then add chopped tomatoes (fresh or canned) along with salt, pepper, and herbs like basil or oregano. Simmer until the sauce thickens, then toss with the cooked pasta.

2. **Pasta Primavera**: 
   - Cook pasta and set aside. In a pan, sauté seasonal vegetables (like bell peppers, zucchini, and broccoli) in olive oil. Add chopped tomatoes, garlic, and herbs, then combine with the cooked pasta.

3. **Caprese Pasta**: 
   - Cook pasta and toss it with diced fresh tomatoes, mozzarella balls, fresh basil, olive oil, balsamic vinegar, salt, and pepper for a refreshing dish.

4. **Pasta Arrabbiata**: 
   - Sauté garlic and red pepper flakes in olive oil, add chopped tomatoes, and simmer. Toss with cooked pasta and finish with fresh parsley or basil.

5. **Pasta B